In [ ]:
import os
import random
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import GridSearchCV 

def normArr(iArr):
    for a,row in enumerate(iArr):
        iArr[a] = (row-np.mean(row))/np.std(row)
    for c in range(np.shape(iArr)[1]):
        col = iArr[:,c]
        iArr[:,c] = (col-np.mean(col))/np.std(col)
    return iArr

def genSpikingArr(trialCount,unitCount,firingRateMax):
    spikeArr = np.random.uniform(0,firingRateMax,size=(trialCount,unitCount))
    return normArr(spikeArr)

def createLabels(trialCount):
    return [random.choice([0,1]) for i in range(trialCount)]

def genAccArr(normedArr,behReal):
    realAcc = []
    for i in range(np.shape(normedArr)[0]):
        for j in range(np.shape(normedArr)[1]):
            if np.isnan(normedArr[i,j]):
                normedArr[i,j] = np.random.uniform(-3,3)
    looR = LeaveOneOut()
    looR.get_n_splits(normedArr)
    for trainInds,testInd in looR.split(normedArr):
        xTrainR,xTestR = normedArr[trainInds],normedArr[testInd]
        yTrainR,yTestR = behReal[trainInds],behReal[testInd]
        param_grid = {"kernel": ["poly"],
              "degree": [1,2,3,4,5,6,7,8,9,10,11,12],
              "coef0": [1],
              "C": [.0001,.001,.01,.1,1,10,100,1000,10000],
              "gamma": ["auto", "scale",.0001,.001,.01,.1,1,10,50,100,125,250,500,750,1000,2500,5000,7500,10000],
              "tol": [.0001,.001,.01,.1,1,10,100,1000,10000],
              "cache_size": [10000],
              "max_iter": [10000]} 
        grid = GridSearchCV(SVC(), param_grid,refit=True,verbose=False)
        grid.fit(xTrainR,yTrainR)
        yPred = grid.predict(xTestR)
        if yPred == yTestR: realAcc.append(1)
        else: realAcc.append(0)
    return round(np.sum(realAcc)/len(realAcc),4)

trialNumber = 50
unitNumber = 70
maxFiringRate = 100
normedSpikingArr = genSpikingArr(trialNumber,unitNumber,maxFiringRate)
behavior = createLabels(trialNumber)
acc = genAccArr(normedSpikingArr,behavior)
